# Fase 2

In [2]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  


# Código para cargar y concatenar los resultados de las extracciones de la Fase 1

In [56]:
# Trasladar todas las tuplas del diccionario a una sola lista
# Paso 1: guardar datos de cada extracción en variables 

extracion_maira = pd.read_json('CSV_JSON/datos_filtrados_2010_2024.json')
extracion_yami = pd.read_json('CSV_JSON/datos_filtrados_Yami_2010_2024.json')
extracion_carolina = pd.read_json('CSV_JSON/datos_filtrados_2010_2024_carolina.json')
extracion_laura = pd.read_json('CSV_JSON/datos_filtrados_2010_2024_laura.json')


In [57]:
# Paso 2: Creamos una función para iterar por cada variable para guardar los datos en listas de tuplas

def itera_variables(extracion):
    anios = [i for i in range(2010, 2025)]
    generos = ['Action', 'Comedy', 'Drama']
    resultados_tuplas = []

    for anio in anios:
        for genero in generos:
            for tupla in extracion[anio][genero]:
                resultados_tuplas.append(tuple(tupla))

    return resultados_tuplas

In [58]:
# Concatenamos todas las listas
resultado_tuplas_total = itera_variables(extracion_maira) + itera_variables(extracion_yami) + itera_variables(extracion_carolina) + itera_variables(extracion_laura)

In [61]:
len(resultado_tuplas_total)

17395

In [60]:
resultado_tuplas_total[:10]

[('Movie', 'Tekken', 2010, 3, 'tt0411951'),
 ('Movie', 'Venus & Vegas', 2010, 11, 'tt0423474'),
 ('Movie', 'The A-Team', 2010, 6, 'tt0429493'),
 ('Movie', 'Segurança Nacional', 2010, 5, 'tt0439801'),
 ('Movie', 'Scott Pilgrim vs. the World', 2010, 11, 'tt0446029'),
 ('Movie', 'Taken by Force', 2010, None, 'tt0465637'),
 ('Movie', 'Guns, Drugs and Dirty Money', 2011, None, 'tt0464032'),
 ('Movie', 'Komaram Puli', 2010, 9, 'tt0464162'),
 ('Movie', 'Prince of Persia: The Sands of Time', 2010, 5, 'tt0473075'),
 ('Movie', 'Unstoppable', 2010, 12, 'tt0477080')]

In [62]:
# Verificar que no haya elementos repetidos
# Crear un conjunto vacío
elementos_vistos = set()

# Verificar duplicados
contador = 0
for elemento in resultado_tuplas_total:
    if elemento in elementos_vistos:
        contador += 1
        resultado_tuplas_total.remove(elemento)
    elementos_vistos.add(elemento)
    
print(f"Se ha encontrado {contador} elementos repetidos")
print(f"Tras remover las tuplas repetidas, nuestra lista se quedó con {len(resultado_tuplas_total)} elementos.")

Se ha encontrado 2401 elementos repetidos
Tras remover las tuplas repetidas, nuestra lista se quedó con 14994 elementos.


In [66]:
# Guardar lista final de manera permanente
df_final = pd.DataFrame(resultado_tuplas_total)
df_final[2] = df_final[2].astype('Int64')
df_final[3] = df_final[3].astype('Int64')
df_final.to_csv('resultado_tuplas_total.csv', index=False)
df_final.to_json('resultado_tuplas_total.json')

# Código para recuperar la lista final de tuplas

In [5]:
resultado_tuplas_total = pd.read_csv('resultado_tuplas_total.csv', header=None) # Ejecutar a partir de aquí

In [6]:
resultado_tuplas_total = [tuple(fila) for fila in resultado_tuplas_total.values]
resultado_tuplas_total = resultado_tuplas_total[1:]

In [7]:
len(resultado_tuplas_total)

14994

In [347]:
for peli in resultado_tuplas_total[700:]: # Prueba: Iterando para extraer el id_imdb y el nombre 
    id_imdb = peli[-1]
    nombre = peli[1]
    print(id_imdb)
    print(nombre)
    break
    

tt0829150
Dracula Untold


In [333]:
search_tomato = id_imdb + ' ' + nombre + ' ' + 'tomatometer' # Creación de una variable con el contenido de la búsqueda para Rotten Tomatoes
search_tomato

'tt0763831 A Thousand Words tomatometer'

In [334]:
# Creación de la variables que vamos a usar en la extración con Selenium
url_imdb = "https://www.imdb.com/"
url_rotten = "https://www.rottentomatoes.com/"
url_google = 'https://www.google.com/'
selector_cookies_imdb = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"
selector_cookies_google = "#L2AGLb > div"

In [335]:
# Creacción de las listas donde vamos a almacenar los resultados de la extración.
resultados_completos = []
resultados_sin_tomatometer = []
not_found = []

In [336]:
# Creamos una función para abrir los navegadores 
def abrir_navegador(driver, url, selector):
    driver = webdriver.Chrome() # Inicializa el navegador Chrome
    sleep(3)
    driver.maximize_window() # Maximiza la ventana
    driver.get(url) # Navega a la url pasada por parámetro
    sleep(3)
    try:
        driver.find_element("css selector", selector).click() # Intentamos aceptar las cookies
        sleep(3)
    except: # Si salta error, imprime un mensaje avisando.
        print("No ha sido posible aceptar las cookies, puede que no saltó el aviso o que se haya cambiado el selector.")
    
    return driver # retorna el driver que será usado para las demás tareas

In [337]:
# Creamos listas con selectores css para hacer iteraciones
selector_imdb_rating = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-1f50b7c-1.bfrqUc > div > div:nth-child(1) > a > span > div > div.sc-eb51e184-0.kFvAju > div.sc-eb51e184-2.czkfBq > span.sc-eb51e184-1.cxhhrI"]
selector_direccion = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(1) > div > ul > li > a", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(1) > div > ul > li > a"]
selector_guionista = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(2) > div > ul > li > a", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(2) > div > ul"]
selector_argumento = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p > span.sc-2d37a7c7-2.PeLXr", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > p > span.sc-2d37a7c7-2.PeLXr"]
selector_duracion = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > ul > li:nth-child(3)", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > ul > li:nth-child(2)"]
selector_nombre = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > h1 > span"]
selector_genero = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.ipc-chip-list--baseAlt.ipc-chip-list.ipc-chip-list--nowrap.sc-2d37a7c7-4.kEPwNU > div.ipc-chip-list__scroller", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.ipc-chip-list--baseAlt.ipc-chip-list.ipc-chip-list--nowrap.sc-2d37a7c7-4.kEPwNU > div.ipc-chip-list__scroller > a"]
selector_primer_result_ggle = ["#rso > div:nth-child(1) > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3", "#rso > div.MjjYud > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3"]
selector_tomatometer = ["#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text", "#rso > div.MjjYud > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3", "#rso > div:nth-child(1) > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3"]

In [338]:
# Creamos una función para iterar por los selectores haciendo los try except para extraer datos de imdb
def extrae_datos(lista_selector, driver, id_imdb=id_imdb, nombre=nombre):
    for selector in lista_selector:
        try:
            dato = driver.find_element("css selector", selector).text
            sleep(2)
            return dato 
        except Exception as e:
            excepcion = e

    error = f"Faltan datos de imdb para la peli: {nombre} id_imdb: {id_imdb}, error de tipo: {excepcion}."
    not_found.append(error)
    dato = None
    return dato

In [339]:
# Creamos una función para pinchar en el primer resultado de google
def clica_primer_result_ggle(lista_selector, driver, id_imdb, nombre):
    try:
        search_tomato = id_imdb + ' ' + nombre + ' ' + 'tomatometer'
        driver.find_element("css selector", "#APjFqb").send_keys(search_tomato, Keys.ENTER) # Pasamos el string al buscador de google
        sleep(7)
        for selector in lista_selector:
            try:                                
                sleep(7)                            
                driver.find_element("css selector", selector).click() # Pinchamos en el primer resultado de la búsqueda de google
                sleep(4)
                return True
            except: # Exception as e:
                # excepcion = e
                pass
    except:
        pass
    
    # error = f"No hay tomatometer para la peli {nombre} con id_imdb {id_imdb}, error de tipo: {excepcion}."
    # not_found.append(error)
    driver.back()
    sleep(5)
    return False

In [340]:
# Creamos una función para aceptar cookies en el navegador de google
def acepta_cookies(driver, on_off):
    try:
        if on_off:
            driver.find_element("css selector", "#onetrust-accept-btn-handler").click() # Acepta cookies
            on_off = False
            sleep(4)
    except:
        pass
        # print("No hemos clicado en el aviso de cookies de google: No hubo aviso, el selector ha cambiado o no hubo tiempo suficiente para terminar de cargar la página.") 

In [341]:
# Creamos una función para extraer el tomatometer
def extrae_tomato(lista_selector, driver, id_imdb, nombre):
    for selector in lista_selector:
        try:                                                   
            tomatometer = driver.find_element("css selector", selector).text # Extrae el Rating tomatometer
            sleep(7)
            driver.back()
            sleep(7)
            driver.back()
            sleep(7)
            return tomatometer
        except Exception as e:
            excepcion = e
            
    tomatometer = None
    error = f"No hay tomatometer para la peli {nombre} con id_imdb {id_imdb}, error de tipo: {excepcion}."
    not_found.append(error)
    driver.back()
    sleep(5)
    driver.back()
    sleep(5)
    return tomatometer

In [350]:
# Llamamos a todas las funciones para extraer los datos

driver1 = None
driver1 = abrir_navegador(driver1, url_imdb, selector_cookies_imdb)

driver2 = None
driver2 = abrir_navegador(driver2, url_google, selector_cookies_google)

contador = 0 # Creamos un contador para saber por que iteración vamos
on_off = True # Creamos un botón de on off para pasar el valor a la función acepta_cookies

'''
Maíra: 0:50, 200:400 ok, next = 700:1000
Yami: 50:100
Laura: 100:150
Carolina: 190:200
'''

# Iterar por los ids_imdbs para pasarlos por el campo search de la web de imdb:
for peli in resultado_tuplas_total[700:1000]:
    id_imdb = peli[-1]
    nombre = peli[1]

    # Pasamos el id_imdb al buscador de imdb
    driver1.find_element("css selector", "#suggestion-search").send_keys(id_imdb, Keys.ENTER)
    sleep(10)

    # Extraímos los datos de imdb
    imdb_rating = extrae_datos(selector_imdb_rating, driver1)
    direccion = extrae_datos(selector_direccion, driver1)
    guionista = extrae_datos(selector_guionista, driver1)
    argumento = extrae_datos(selector_argumento, driver1)
    genero = extrae_datos(selector_genero, driver1)
    duracion = extrae_datos(selector_duracion, driver1)
    nombre = extrae_datos(selector_nombre, driver1)

    # Extraímos el tomatometer
    pinchar_google = clica_primer_result_ggle(selector_primer_result_ggle, driver2, id_imdb, nombre)
    if pinchar_google:
        acepta_cookies(driver2, on_off)
        tomatometer = extrae_tomato(selector_tomatometer, driver2, id_imdb, nombre)
    else:
        tomatometer = None
        
    # Creamos la tupla y la añadimos a la lista que corresponda (completa o sin tomatometer)
    if tomatometer:
        tupla = imdb_rating, tomatometer, direccion, guionista, argumento, genero, duracion, nombre
        if None in tupla:
            contador += 1
            print(f"La tupla de la peli {nombre} no está completa: {tupla} Vamos por la iteración: {contador}")
        else:
            resultados_completos.append(tupla)
            contador += 1
            print(f"Hemos añadido la peli {nombre} a la lista completa. Vamos por la iteración: {contador}.")
    else:
        tupla = imdb_rating, direccion, guionista, argumento, genero, duracion, nombre
        if None in tupla:
            contador += 1
            print(f"La tupla de la peli {nombre} no está completa: {tupla} Vamos por la iteración: {contador}")
        else:
            tupla = imdb_rating, tomatometer, direccion, guionista, argumento, genero, duracion, nombre
            resultados_sin_tomatometer.append(tupla)
            contador += 1
            print(f"Hemos añadido la peli {nombre} a la lista sin tomatometer. Vamos por la iteración: {contador}.")

driver1.close()
driver2.close()

Hemos añadido la peli Dracula Untold a la lista sin tomatometer. Vamos por la iteración: 1.
Hemos añadido la peli Electric Slide a la lista sin tomatometer. Vamos por la iteración: 2.
Hemos añadido la peli Bhopal: A Prayer for Rain a la lista sin tomatometer. Vamos por la iteración: 3.
Hemos añadido la peli Suite Française a la lista sin tomatometer. Vamos por la iteración: 4.
Hemos añadido la peli Love & Mercy a la lista sin tomatometer. Vamos por la iteración: 5.
Hemos añadido la peli A Merry Christmas Miracle a la lista sin tomatometer. Vamos por la iteración: 6.
Hemos añadido la peli Coffee Sex You a la lista sin tomatometer. Vamos por la iteración: 7.
Hemos añadido la peli Collection a la lista sin tomatometer. Vamos por la iteración: 8.
Hemos añadido la peli Avenge a la lista sin tomatometer. Vamos por la iteración: 9.
La tupla de la peli Cocoon no está completa: (None, 'Chavi Khare', 'Chavi Khare', '', 'Short', None, 'Cocoon') Vamos por la iteración: 10
La tupla de la peli Zahte

In [353]:
len(resultados_sin_tomatometer)

167

In [354]:
len(resultados_completos)

5

In [319]:
resultados_completos[-1]

('5.9',
 '40%',
 'Anthony Hemingway',
 'John Ridley',
 'A crew of African American pilots in the Tuskegee training program, having faced segregation while kept mostly on the ground during World War II, are called into duty under the guidance of Col. A.J. Bullard.',
 'Action\nDrama\nHistory',
 '2h 5m',
 'Red Tails')

In [145]:
# Se tarda aproximadamente 33 minutos para extraer 50 peliculas.
# Se tarda aproximadamente 67 minutos para extraer 100 peliculas.

In [355]:
# Guardar los datos de manera permanente
df_resultados_completos = pd.DataFrame(resultados_completos)
df_resultados_sin_tomatometer = pd.DataFrame(resultados_sin_tomatometer)
# df_errores = pd.DataFrame(not_found)
df_resultados_completos.to_csv('resultados_completos_F2_maira_544-1000.csv') # Cambiar nombre !!! 
df_resultados_sin_tomatometer.to_csv('resultados_sin_tomatometer_F2_maira_544-1000.csv')
# df_errores.to_csv('errores_F2_maira_544-1000.csv')
df_resultados_completos.to_json('resultados_completos_F2_maira_544-1000.json') 
df_resultados_sin_tomatometer.to_json('resultados_sin_tomatometer_F2_maira_544-1000.json')
# df_errores.to_json('errores_F2_maira_544-1000.json')

In [373]:
# Función para concatenar las extracciones en un solo csv

# def concatena_csv(archivo_csv):
df_completo = pd.read_csv('CSV_JSON_F2/Maira/resultados_sin_tomatometer_F2_maira_0-50_200-543.csv', index_col=0)


In [374]:
df_completo_1 = pd.read_csv('resultados_sin_tomatometer_F2_maira_544-1000.csv', index_col=0)


In [375]:
df_completo_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       167 non-null    float64
 1   1       0 non-null      float64
 2   2       167 non-null    object 
 3   3       167 non-null    object 
 4   4       160 non-null    object 
 5   5       167 non-null    object 
 6   6       167 non-null    object 
 7   7       167 non-null    object 
dtypes: float64(2), object(6)
memory usage: 11.7+ KB


In [376]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       172 non-null    float64
 1   1       0 non-null      float64
 2   2       172 non-null    object 
 3   3       172 non-null    object 
 4   4       168 non-null    object 
 5   5       172 non-null    object 
 6   6       172 non-null    object 
 7   7       172 non-null    object 
dtypes: float64(2), object(6)
memory usage: 12.1+ KB


In [377]:
df_concatenado = pd.concat([df_completo, df_completo_1], ignore_index=True)

In [379]:
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       339 non-null    float64
 1   1       0 non-null      float64
 2   2       339 non-null    object 
 3   3       339 non-null    object 
 4   4       328 non-null    object 
 5   5       339 non-null    object 
 6   6       339 non-null    object 
 7   7       339 non-null    object 
dtypes: float64(2), object(6)
memory usage: 21.3+ KB


In [380]:
df_concatenado.to_csv('resultados_sintomato_final_maira.csv')

In [ ]:
# Puntuacion de IMDB (en caso de que la tenga).

# Puntuación de Rotten Tomatoes (Tomatometer).

# Dirección (director/a o directore/as de cada película).

# Gionistas (de cada película).

# Argumento.

# Duración. >> Tal cuál está 

# Nombre de la película

# Genero !!! 

# NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

# [(7.7, 77, "Richard Donner", ["Chris ColumbusSteven", "Spielberg"], "Los Goonies son un grupo de amigos que viven en Goon Docks, Astoria, pero sus casas han sido compradas y van a ser demolidas. Sin embargo, vivirán su última aventura en busca de un tesoro que pueda salvar el barrio.", "Aventura", "1h 54min", "Los Gonnies"), ...]

In [ ]:
# CSV_JSON_F2/todas_extracciones/resultados_completos_F2_carolina_0-39.csv
# CSV_JSON_F2/todas_extracciones/resultados_completos_F2_carolina_190-200.csv
# CSV_JSON_F2/todas_extracciones/resultados_completos_F2_yami_50-100.csv
# CSV_JSON_F2/todas_extracciones/resultados_completos_F2_yami_2000-2200.csv
# CSV_JSON_F2/todas_extracciones/resultados_completos_F2_yami_2200-2400.csv